In [8]:
import pandas as pd
pd.__version__

'0.24.2'

In [10]:
pépé=0
pépé+1

1

In [24]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName('lololand').getOrCreate()
spark

In [31]:
spark.stop()

In [32]:
import pyspark
sc = pyspark.SparkContext(appName="rien")

import urllib.request as url
f = url.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [33]:
raw_data.count()

494021

In [35]:
raw_data.take(2)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.']

In [38]:
type(raw_data)

pyspark.rdd.RDD

In [40]:
# parse data
csv_data = raw_data.map(lambda x: x.split(","))

# separate into different RDDs
normal_csv_data = csv_data.filter(lambda x: x[41]=="normal.")
attack_csv_data = csv_data.filter(lambda x: x[41]!="normal.")
print(normal_csv_data.count())
print(attack_csv_data.count())

97278
396743


In [54]:
normal_csv_data.map(lambda x: int(x[0]))

PythonRDD[10] at RDD at PythonRDD.scala:53

In [56]:
normal_csv_data.map(lambda x: int(x[0])).reduce(lambda x, y: x + y)

21075991

In [57]:
attack_csv_data.map(lambda x: int(x[0])).reduce(lambda x, y: x + y)

2626792

voir https://github.com/jadianes/spark-py-notebooks/blob/master/nb5-rdd-aggregations/nb5-rdd-aggregations.ipynb

In [61]:
attack_csv_data.map(lambda x: int(x[0])).aggregate(
    (0,0),
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), 
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))

(2626792, 396743)

In [23]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

3.1414384


In [17]:
sc = pyspark.SparkContext(appName="rien")

In [18]:
sc

<SparkContext master=local[*] appName=rien>

In [19]:
sc.stop()

In [41]:
def t0 (*a):
    for x in a:
        print(x)

t0(1,2,3)

1
2
3


In [48]:
def t0 (**a):
    for k,v in a.items():
        print(k,v)
    print (a['xx'])

t0(a=1, b=3, xx='boo')

a 1
b 3
xx boo
boo


In [70]:
l=['a','b','c']
map(lambda x: x+'x', l)

# SVD

In [72]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

sc = pyspark.SparkContext(appName="matrix")
rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

mat = RowMatrix(rows)

# Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(5, computeU=True)
U = svd.U
s = svd.s
V = svd.V

In [91]:
x=Vectors.sparse(5, {1: 1.0, 3: 7.0})
x.toArray

<bound method SparseVector.toArray of SparseVector(5, {1: 1.0, 3: 7.0})>

In [87]:
rows.collect()

[SparseVector(5, {1: 1.0, 3: 7.0}),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0])]

In [81]:
U.numCols(), U.numRows()

(5, 3)

In [83]:
s

DenseVector([13.0293, 5.3686, 2.533, 0.0, 0.0])

In [85]:
V

DenseMatrix(5, 5, [-0.3128, -0.0298, -0.1221, -0.7185, -0.6084, 0.3117, -0.1713, 0.1526, ..., 0.2383, -0.0209, -0.4626, -0.0745, 0.9735, -0.0345, -0.1391, 0.1618], 0)

In [93]:
collected = U.rows.collect()
print("U factor is:")
for vector in collected:
    print(vector)
print("Singular values are: %s" % s)
print("V factor is:\n%s" % V)
sc.stop()

U factor is:
[-0.38829130511665644,-0.9198099362554474,-0.056387441301709175,9.313225746154785e-09,0.0]
[-0.5301719995198351,0.2730185511901228,-0.8027319114319463,0.0,0.0]
[-0.7537556058139434,0.2817987790459642,0.5936682026454339,1.4901161193847656e-08,1.4901161193847656e-08]
Singular values are: [13.029275535600473,5.368578733451684,2.5330498218813755,6.323166049206486e-08,2.0226934557075942e-08]
V factor is:
DenseMatrix([[-0.31278534,  0.31167136,  0.30366911,  0.8409913 , -0.07446478],
             [-0.02980145, -0.17133211, -0.02226069,  0.14664984,  0.97352733],
             [-0.12207248,  0.15256471, -0.95070998,  0.23828799, -0.03452092],
             [-0.71847899, -0.68096285, -0.0172245 , -0.02094998, -0.13907533],
             [-0.60841059,  0.62170723,  0.05606596, -0.46260933,  0.16175873]])


# TP

In [99]:
from random import sample, random
sc = pyspark.SparkContext(appName="tp")
dat_1=sc.parallelize(range(1, 5+1))
coll = dat_1.collect()
print ("Elements in RDD -> ",coll)
dat_2=sc.parallelize(range(1, 5+1))
coll = dat_2.collect()
print ("Elements in RDD -> ",coll)
words = sc.parallelize (["scala", "java", "hadoop", "spark", "akka","spark vs hadoop","pyspark", "pyspark and spark"])
coll = words.collect()
print ("Elements in RDD -> %s" % coll)


Elements in RDD ->  [1, 2, 3, 4, 5]
Elements in RDD ->  [1, 2, 3, 4, 5]
Elements in RDD -> ['scala', 'java', 'hadoop', 'spark', 'akka', 'spark vs hadoop', 'pyspark', 'pyspark and spark']


In [103]:
words_map = words.map(lambda x: (x, 1))
mapping = words_map.collect()
print ("Key value pair -> %s" % (mapping))
dat_1=dat_1.map(lambda x:(x, sample(["a", "b","c"], 1)[0]))
dat_2=dat_2.map(lambda x:(x, random()))
dat_2.persist()
# we need to cache dat_2 otherwise it will generate random every time dat_2 is invoked

Key value pair -> [('scala', 1), ('java', 1), ('hadoop', 1), ('spark', 1), ('akka', 1), ('spark vs hadoop', 1), ('pyspark', 1), ('pyspark and spark', 1)]


PythonRDD[12] at RDD at PythonRDD.scala:53

In [104]:
dat_1.flatMap(lambda x:x).collect()

[1, 'c', 2, 'c', 3, 'c', 4, 'c', 5, 'c']

In [105]:
dat_1.filter(lambda x:x[0] in [1,2,3]).collect()

[(1, 'c'), (2, 'c'), (3, 'c')]

In [106]:
dat=dat_1.union(dat_2)
dat.collect()

[(1, 'c'),
 (2, 'c'),
 (3, 'c'),
 (4, 'c'),
 (5, 'c'),
 (1, 0.8587614275356122),
 (2, 0.21188194931111426),
 (3, 0.3239564282546593),
 (4, 0.5105798135271437),
 (5, 0.019554884872437728)]

In [107]:
dat_1.intersection(dat_2).collect()

[]

In [108]:
dat_1.union(dat_1).collect()

[(1, 'c'),
 (2, 'c'),
 (3, 'c'),
 (4, 'c'),
 (5, 'c'),
 (1, 'c'),
 (2, 'c'),
 (3, 'c'),
 (4, 'c'),
 (5, 'c')]

In [109]:
dat_1.union(dat_1).distinct().collect()

[(4, 'c'), (5, 'c'), (2, 'c'), (3, 'c'), (1, 'c')]

In [111]:
dat_1.groupByKey().collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x7f4aaee64630>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7f4aaedaa7b8>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f4aaedaaac8>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7f4aaedaaf98>),
 (5, <pyspark.resultiterable.ResultIterable at 0x7f4aaedaa7f0>)]

In [112]:
dat_1.groupByKey().collect()[0]

(1, <pyspark.resultiterable.ResultIterable at 0x7f4aaee9d6a0>)

In [113]:
dat_1.groupByKey().collect()[0][1]

In [114]:
dat_1.groupByKey().collect()[0][1].data

['c']

In [115]:
# Here I union dat_2 with itself and then reduce by key to get the sum of all values of each key. So the results we get below should be the double of the values above
dat_2.union(dat_2).reduceByKey(lambda x,y:x+y).collect()

[(1, 1.7175228550712245),
 (2, 0.4237638986222285),
 (3, 0.6479128565093186),
 (4, 1.0211596270542873),
 (5, 0.039109769744875456)]

In [116]:
dat_2.union(dat_2).aggregateByKey(100, lambda a,b:a+b, lambda x,y:x+y).collect()

[(1, 201.71752285507122),
 (2, 200.42376389862224),
 (3, 200.64791285650932),
 (4, 201.0211596270543),
 (5, 200.03910976974487)]

In [119]:
dat_3=sc.parallelize(list("1223334444")*10000000)
dat_3=dat_3.map(lambda x:(x, random()))
dat_3=dat_3.aggregateByKey((0.0,0), lambda a,b:(a[0]+b, a[1]+1), lambda rdd1,rdd2:(rdd1[0]+rdd2[0], rdd1[1]+rdd2[1]))
dat_3.collect()

[('4', (19999239.954979904, 40000000)),
 ('1', (4998945.601639987, 10000000)),
 ('3', (14998390.881799575, 30000000)),
 ('2', (10000631.914942238, 20000000))]

In [120]:
%%bash
ls

data.txt
form_spark.ipynb
kafkaReceiveAndSaveToCassandraPy-Copy1.ipynb
kafkaSendDataPy-Copy1.ipynb
kddcup.data_10_percent.gz
names.txt
notes_form_adm
nyse_daily_prices_k.csv
nyse_dividends_k.csv
stocks.csv
testspark.ipynb
visits.txt


In [145]:
sql=pyspark.sql.SQLContext(sc)
df= sql.read.format('csv')\
    .option('inferSchema', True)\
    .option('header', True)\
    .option('sep',',')\
    .load('nyse_daily_prices_k.csv')

In [146]:
df.createOrReplaceTempView('tmp')

In [147]:
df.printSchema()

root
 |-- exchange: string (nullable = true)
 |-- stock_symbol: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- stock_price_open: double (nullable = true)
 |-- stock_price_high: double (nullable = true)
 |-- stock_price_low: double (nullable = true)
 |-- stock_price_close: double (nullable = true)
 |-- stock_volume: integer (nullable = true)
 |-- stock_price_adj_close: double (nullable = true)



In [152]:
df2= sql.sql('select exchange, stock_price_high from tmp')
df2.printSchema()

root
 |-- exchange: string (nullable = true)
 |-- stock_price_high: double (nullable = true)



In [161]:
help(df2.rdd)

Help on RDD in module pyspark.rdd object:

class RDD(builtins.object)
 |  RDD(jrdd, ctx, jrdd_deserializer=AutoBatchedSerializer(PickleSerializer()))
 |  
 |  A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.
 |  Represents an immutable, partitioned collection of elements that can be
 |  operated on in parallel.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      Return the union of this RDD and another one.
 |      
 |      >>> rdd = sc.parallelize([1, 1, 2, 3])
 |      >>> (rdd + rdd).collect()
 |      [1, 1, 2, 3, 1, 1, 2, 3]
 |  
 |  __getnewargs__(self)
 |  
 |  __init__(self, jrdd, ctx, jrdd_deserializer=AutoBatchedSerializer(PickleSerializer()))
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  aggregate(self, zeroValue, seqOp, combOp)
 |      Aggregate the elements of each partition, and then the results for all
 |      the partitions, using a given combine fun

In [160]:
df2.rdd.collect(5)

TypeError: collect() takes 1 positional argument but 2 were given

In [156]:
df2.show(5)

+--------+----------------+
|exchange|stock_price_high|
+--------+----------------+
|    NYSE|           27.92|
|    NYSE|           28.16|
|    NYSE|           28.31|
|    NYSE|           28.89|
|    NYSE|            29.1|
+--------+----------------+
only showing top 5 rows



In [188]:
from pyspark.sql.functions import upper, length, udf
from pyspark.sql.types import DoubleType, StringType

In [200]:
def test(x,y):
    return str((x+y)/3)[:5]+'aze'

In [201]:
testudf= udf(test, StringType())
#sql.udf.register('testudf',testudf)

In [202]:
df.withColumn('test',testudf(df.stock_price_high, df.stock_price_low)).show(7)

+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+--------+
|exchange|stock_symbol|               date|stock_price_open|stock_price_high|stock_price_low|stock_price_close|stock_volume|stock_price_adj_close|    test|
+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+--------+
|    NYSE|         KOP|2010-02-08 00:00:00|           27.92|           27.92|           27.0|            27.02|      101500|                27.02|18.30aze|
|    NYSE|         KOP|2010-02-05 00:00:00|           27.98|           28.16|          26.96|            27.99|      116300|                27.99|18.37aze|
|    NYSE|         KOP|2010-02-04 00:00:00|            28.3|           28.31|          27.68|            27.96|      163300|                27.96|18.66aze|
|    NYSE|         KOP|2010-02-03 00:00:00|           28.67|    

In [98]:
sc.stop()